In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# standard lib
import os, pwd, sys, json, yaml, atexit, tempfile, inspect
from pathlib import Path

# for data-science
import pandas as pd, numpy as np, quadfeather
from pyarrow import feather

# for plotting
import matplotlib as mpl, matplotlib.pyplot as plt, seaborn as sns

# for cellular-data
import scprep, scanpy as sc, anndata as ad

In [3]:
from featherplot.utils import MockSingleCellData, AnnDataProcessor, QuadFeatherRenamer
from featherplot.utils import SeriesToChannel, DataFrameToMetadata

In [4]:
mocker = MockSingleCellData()
adata = mocker.adata

In [5]:
adata

AnnData object with n_obs × n_vars = 1000 × 100
    obs: 'barcodes', 'conditions'
    var: 'is_hvg'
    obsm: 'X_mock'
    layers: 'X_norm'

### Create Processor
> this will help us extract the embedding layer and the gene expression layer

In [6]:
pipe = AnnDataProcessor(adata, 'X_mock', 'X_norm')

#### sidecars

Deepscatter calls additional columns `sidecars`, in our case those are the columns of gene expression. We place these values in `df_s`.

In [7]:
df_s = pipe.get_sidecars()
df_s.head()

gene_symbols,gene_symbol 0,gene_symbol 1,gene_symbol 2,gene_symbol 3,gene_symbol 4,gene_symbol 5,gene_symbol 6,gene_symbol 7,gene_symbol 8,gene_symbol 9,...,gene_symbol 90,gene_symbol 91,gene_symbol 92,gene_symbol 93,gene_symbol 94,gene_symbol 95,gene_symbol 96,gene_symbol 97,gene_symbol 98,gene_symbol 99
barcodes,,,,,,,,,,,,,,,,,,,,,
barcode 0,0.647239,0.086998,-0.154814,-1.654152,-1.685163,0.105083,-0.844099,-0.896430,-0.323254,-0.533175,...,0.916992,1.529056,-0.587797,-0.947348,0.151334,0.052068,0.688556,1.029177,-0.275412,2.362747
barcode 1,1.404417,0.137621,1.675856,-0.104808,1.076974,-0.707360,-0.112802,1.591732,-1.140162,0.888240,...,-0.069031,-1.743513,1.011159,1.468735,0.492341,1.107176,0.589416,1.465082,1.530432,-0.479527
barcode 2,-1.060918,0.236627,0.077683,1.249768,0.737647,1.218718,1.024711,-0.675470,2.130783,-1.216797,...,-1.449032,-1.151383,-1.106516,-0.583469,1.075358,-0.314728,-0.191897,-0.159697,-1.726697,-0.521543
barcode 3,1.448654,-1.494581,0.223879,2.088154,1.116556,0.727096,1.298132,-0.310056,-0.362844,-1.115445,...,-0.463154,-2.504597,0.611575,0.509425,0.579520,1.007162,0.191839,-0.782431,1.152722,-0.741739
barcode 4,-0.369716,1.675644,-0.657181,-1.543317,1.660483,-0.371225,-0.303722,0.795800,0.496244,0.178755,...,0.785304,-0.317869,-0.804447,0.890166,1.153516,2.804246,-0.314702,0.053635,0.896291,-0.340139


#### points

If our gene expression features are called `sidecars`, then what is the embedding layer called? Well it is just the "points" of the plot, so we will store these values in `df_p`.

**NOTE**: we also store conditions with `df_p` as whatever is in this DataFrame will be loaded by `Deepscatter` automatically. 

In [8]:
df_p = pipe.get_embedding()
df_p = df_p.join(pipe.adata.obs.conditions)
df_p.head()

,MOCK_0,MOCK_1,MOCK_2,conditions
barcodes,,,,
barcode 0,-1.752125,0.279810,-1.627978,condition 0
barcode 1,-0.423364,-0.658018,-1.544936,condition 1
barcode 2,-2.045956,1.201831,0.486148,condition 2
barcode 3,0.444498,0.439598,1.582178,condition 3
barcode 4,-1.920090,-1.453035,0.247593,condition 0


#### Combined
Now we combine `df_p` (points + condition) with `df_s` ("sidecars" i.e. gene expression). This is necessary as for script later on where we need to add the sidecars to already the `quadfeather`-ed (tiled) point data. 

In [9]:
df_all = df_p.join(df_s)
df_all.head()

,MOCK_0,MOCK_1,MOCK_2,conditions,gene_symbol 0,gene_symbol 1,gene_symbol 2,gene_symbol 3,gene_symbol 4,gene_symbol 5,...,gene_symbol 90,gene_symbol 91,gene_symbol 92,gene_symbol 93,gene_symbol 94,gene_symbol 95,gene_symbol 96,gene_symbol 97,gene_symbol 98,gene_symbol 99
barcodes,,,,,,,,,,,,,,,,,,,,,
barcode 0,-1.752125,0.279810,-1.627978,condition 0,0.647239,0.086998,-0.154814,-1.654152,-1.685163,0.105083,...,0.916992,1.529056,-0.587797,-0.947348,0.151334,0.052068,0.688556,1.029177,-0.275412,2.362747
barcode 1,-0.423364,-0.658018,-1.544936,condition 1,1.404417,0.137621,1.675856,-0.104808,1.076974,-0.707360,...,-0.069031,-1.743513,1.011159,1.468735,0.492341,1.107176,0.589416,1.465082,1.530432,-0.479527
barcode 2,-2.045956,1.201831,0.486148,condition 2,-1.060918,0.236627,0.077683,1.249768,0.737647,1.218718,...,-1.449032,-1.151383,-1.106516,-0.583469,1.075358,-0.314728,-0.191897,-0.159697,-1.726697,-0.521543
barcode 3,0.444498,0.439598,1.582178,condition 3,1.448654,-1.494581,0.223879,2.088154,1.116556,0.727096,...,-0.463154,-2.504597,0.611575,0.509425,0.579520,1.007162,0.191839,-0.782431,1.152722,-0.741739
barcode 4,-1.920090,-1.453035,0.247593,condition 0,-0.369716,1.675644,-0.657181,-1.543317,1.660483,-0.371225,...,0.785304,-0.317869,-0.804447,0.890166,1.153516,2.804246,-0.314702,0.053635,0.896291,-0.340139


### QuadFeatherRenamer

Note: `quadfeather` and `deepscatter` are both under active development so things change all the time. At the moment `quadfeather` requires that `x` and `y` be in your DataFrame (it doesn't mind if `z` is there too). So this will handle the renaming of our columns.

In [10]:
qfr = QuadFeatherRenamer(df_all)

In [11]:
df_q, renamed = qfr.rename()
renamed

{'MOCK_0': 'x', 'MOCK_1': 'y', 'MOCK_2': 'z'}

### DataFrameToMetadata
 
`Deepscatter` is a really nice library; however, it also prefers to have its `plotAPI` method called with as much information as possible. This is a bit of a shame as it means that one you load your data with `deepscatter` you can't compute derived properties (e.g. domain of your data to scale the plot, check for what sidecars are availble, etc). 

The solution to this is simple. In order to have this information availble to us, we will just calculate it now (including which columns were renamed) and store it as metadata to use later

In [12]:
d2m = DataFrameToMetadata(
    df_q, 
    include_index=True,
    embedding='x y z conditions'.split(),
    alt_names={v:k for k,v in renamed.items()}
)

In [13]:
succ, fail = d2m.convert()
len(succ), len(fail)

(105, 0)

In [14]:
meta = d2m.to_meta()

## Quadfeather Workflow

Now we can now run through thte `quadfeather` workflow right here in the notebook.

### 0) setup

In [25]:
# dump everything to downloads for easy access
outdir = os.path.expanduser('~/Downloads/featherplot')
qf_dir = os.path.join(outdir, 'tiles')
if not os.path.isdir(qf_dir):
    os.makedirs(qf_dir)


p_file = os.path.join(outdir, 'points.parquet')
# NOTE: we never use s_file
# s_file = os.path.join(outdir, 'extras.parquet')
f_file = os.path.join(qf_dir, 'sidecars.feather')
m_file = os.path.join(outdir, 'meta.yml')


tile_size = 1000

### 1) create tiles

In [24]:
d2m.df.drop(columns=df_s.columns).to_parquet(p_file)
# d2m.df.drop(columns=d2m.embedding).to_parquet(s_file)

In [26]:
!quadfeather --files {p_file} \
             --tile_size {tile_size} \
             --destination {qf_dir}

### 2) make single file

In [30]:
feather.write_feather(d2m.df.drop(columns=d2m.embedding), f_file)

### 3) run `add_sidecars.py`

In [42]:
from featherplot.utils import collapse_user
from featherplot.deepscatter import Tileset

In [ ]:
tileset = Tileset(Path(qf_dir))
tileset.add_sidecars(f_file, d2m.df.index.name)

note we copied `add_sidecars.py` so you can use it directly from this library

```sh
featherplot-py featherplot add-sidecars --help

Usage: featherplot add-sidecars 
[OPTIONS]
--tileset          PATH  Path to the tileset to add sidecars to.
--sidecar          PATH  Path to the new data to add to the tileset.
--key              TEXT  key to use for joining; must exist in both tables
--verbose  -v            Print verbose output.
--help                   Show this message and exit.
```

alternatively you can run the script form wherever you saved it

In [ ]:
!python3 add_sidecars.py --tileset {qf_dir}\
                         --sidecar {f_file} --key {d2m.df.index.name};

### 4) update metadata with directo

In [41]:
meta.keys()

dict_keys(['index', 'n_points', 'embedding', 'sidecars', 'columns_metadata', 'tiles_dir'])

In [45]:
# relative path to tiles
meta['tiles_dir'] = qf_dir.replace(outdir, '')
# full path to tiles
meta['full_path'] = collapse_user(qf_dir)

In [46]:
with open(m_file, 'w') as f:
    f.write(yaml.dump(meta))

### 5) cleanup